### 重置摄像头

In [ ]:
!echo 'jetson' | sudo -S systemctl restart nvargus-daemon && printf '\n'

### 初始化

In [ ]:
import torch
import torchvision
from torch2trt import torch2trt
from torch2trt import TRTModule

TASK="test"

CATEGORIES = ['apex']

device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
model = model.cuda().eval().half()
model.load_state_dict(torch.load(TASK+'/path/model.pth'))

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

torch.save(model_trt.state_dict(), TASK+'/road_model_trt.pth')

model_trt = TRTModule()
model_trt.load_state_dict(torch.load(TASK+'/road_model_trt.pth'))

from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

初始化摄像头

In [ ]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=65)

开始跑

In [ ]:
from utils import preprocess
import numpy as np
import threading
import ipywidgets
import time
import cv2
from jetcam.utils import bgr8_to_jpeg

car.steering_gain = -0.55
car.steering_offset = -0.13
car.throttle_gain = 0.5
# car.throttle = 0.25
speed_widget = ipywidgets.IntText(description='速度', value=20)
run_button = ipywidgets.Button(description='启动')
state = 1
prediction_widget = ipywidgets.Image(format='jpeg', width=224, height=224)

def run(car,camera,speed_widget,run_button,prediction_widget):
    global state
    while True:
        image = camera.read()
        prediction = image.copy()
        image = preprocess(image).half()
        output = model_trt(image).detach().cpu().numpy().flatten()
        dx = output[0]
        dy = output[1]

        dx = int(224 * (dx / 2.0 + 0.5))
        dy = int(224 * (dy / 2.0 + 0.5))

        
        prediction = cv2.circle(prediction, (dx, dy), 8, (255, 0, 0), 3)
        prediction_widget.value = bgr8_to_jpeg(prediction)
        
        if (state == 1):
            if car.throttle>0 :
                car.throttle = -10
            else :
                car.throttle = 0
            car.steering = 0
            time.sleep(0.5)
        else:
            x = float(output[0])
            #car.steering = x * STEERING_GAIN + STEERING_BIAS
            car.steering = x
            car.throttle = speed_widget.value*0.01
            

        
        
execute_thread = threading.Thread(target=run, args=(car,camera,speed_widget,run_button,prediction_widget))

def runclick(c):
    global state
    print(run_button.description)
    if run_button.description == '启动':
        run_button.description = '停止'
        state = 0
    else:
        run_button.description = '启动'
        state = 1

run_button.on_click(runclick)

data_collection_widget = ipywidgets.HBox([
    run_button,
    speed_widget
])
data_display_widget = ipywidgets.VBox([
    prediction_widget,
    data_collection_widget
])

display(data_display_widget)
execute_thread.start()
